In [ ]:
import pandas as pd
BASE_URL = "https://voyager.online/api/staking/validators"
page_size = 50

# Iterate over pages 1 to 3 (adjust the range as needed)
for p in range(1, 4):
    url = f"{BASE_URL}?search=&p={p}&ps={page_size}"
    data = requests.get(url).json()
    print(f"Page {p}:")
    print(data)


In [ ]:
host = "https://voyager.online/api/staking/validators?search=&p=1&ps=20"

result = requests.get(host).json()
result







In [4]:
import requests
import psycopg2

# PostgreSQL connection details - update these with your credentials
conn = psycopg2.connect(
    dbname="mydatabase",
    user="user",
    password="password",
    host="localhost",
    port="5432"  # typically 5432
)
cur = conn.cursor()

# Create the "validators" table if it doesn't exist, using proper numeric types
create_table_query = """
CREATE TABLE IF NOT EXISTS validators (
    address TEXT PRIMARY KEY,
    tokenAddress TEXT,
    isVerified BOOLEAN,
    totalStake NUMERIC,             -- changed from TEXT to NUMERIC
    totalStakePercentage NUMERIC,   -- changed from INTEGER to NUMERIC if decimals are needed
    ownStake NUMERIC,               -- changed from TEXT to NUMERIC
    delegatedStake NUMERIC,         -- changed from TEXT to NUMERIC
    totalDelegators INTEGER,
    revenueShare NUMERIC,           -- changed from INTEGER to NUMERIC if needed
    apr NUMERIC,                    -- changed from INTEGER to NUMERIC if decimals are needed
    rank TEXT,
    name TEXT,
    imgSrc TEXT,
    startTime BIGINT,
    poolAddress TEXT,
    stakerAddress TEXT
);
"""
cur.execute(create_table_query)
conn.commit()

BASE_URL = "https://voyager.online/api/staking/validators"
page_size = 100

# Iterate over pages 1, 2, and 3 (adjust the range as needed)
for p in range(1, 4):
    url = f"{BASE_URL}?search=&p={p}&ps={page_size}"
    response = requests.get(url)
    data = response.json()
    items = data.get("items", [])
    
    # Insert each item into the Postgres table
    for item in items:
        query = """
        INSERT INTO validators (
            address, tokenAddress, isVerified, totalStake, totalStakePercentage,
            ownStake, delegatedStake, totalDelegators, revenueShare, apr, rank,
            name, imgSrc, startTime, poolAddress, stakerAddress
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (address) DO NOTHING;
        """
        values = (
            item.get("address"),
            item.get("tokenAddress"),
            item.get("isVerified"),
            item.get("totalStake"),
            item.get("totalStakePercentage"),
            item.get("ownStake"),
            item.get("delegatedStake"),
            item.get("totalDelegators"),
            item.get("revenueShare"),
            item.get("apr"),
            item.get("rank"),
            item.get("name"),
            item.get("imgSrc"),
            item.get("startTime"),
            item.get("poolAddress"),
            item.get("stakerAddress")
        )
        cur.execute(query, values)
    
    conn.commit()
    print(f"Inserted data from page {p}")

cur.close()
conn.close()

Inserted data from page 1
Inserted data from page 2
Inserted data from page 3


In [3]:
import requests
import psycopg2

# PostgreSQL connection details - update these with your credentials
conn = psycopg2.connect(
    dbname="mydatabase",
    user="user",
    password="password",
    host="localhost",
    port="5432"  # typically 5432
)
cur = conn.cursor()

query_get_avg_delegated_stake_top_10 = """SELECT AVG(delegatedStake) 
FROM (
    SELECT delegatedStake 
    FROM validators 
    ORDER BY delegatedStake DESC
    limit 10
) subquery;"""


cur.execute(query_get_avg_delegated_stake_top_10)
result = cur.fetchall()
print(result[0][0] / 10**18)

cur.close()
conn.close()


15209196.437081446278959945
